In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv
from serpapi import GoogleSearch

from langchain import ConversationChain, LLMChain, OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chains import ConversationChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (ChatPromptTemplate, HumanMessagePromptTemplate,
                               MessagesPlaceholder, PromptTemplate,
                               SystemMessagePromptTemplate)
from langchain.prompts.chat import (ChatPromptTemplate,
                                    HumanMessagePromptTemplate,
                                    SystemMessagePromptTemplate)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv("OPENAI_API_KEY")

# LLMs: Get predictions from a language model

LangChain의 가장 기본적인 구성 요소는 입력값에 대해 LLM을 호출하는 것입니다. 이를 수행하는 간단한 예제를 살펴보겠습니다. 이를 위해, 회사가 무엇을 생산하는지에 기반하여 회사 이름을 생성하는 서비스를 구축한다고 가정해 봅시다. 이를 위해서는 먼저 LLM 래퍼를 가져와야 합니다.

In [3]:
llm = OpenAI(temperature=0.9)

In [4]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Bright Footwear.


# Prompt Templates: Manage prompts for LLMS

일반적으로, LLM을 사용할 때, 사용자 입력을 직접 보내는 것이 아니라, 사용자 입력을 받아 프롬프트(prompt)를 작성한 후 LLM에게 보냅니다. 이 문서에서는 LangChain을 사용하여 프롬프트 템플릿을 정의하는 방법을 설명합니다. 프롬프트 템플릿은 사용자 입력에서 필요한 정보만 추출하여 해당 정보를 이용해 프롬프트를 구성하는 것이 가능합니다.

In [5]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [6]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


# Chains: Combine LLMs and prompts in multi-step workflows

In [7]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
chain.run("colorful socks")

'\n\nRainbow Toe Socks'

# Agents: Dynamically Call Chains Based on User Input

에이전트는 LLM(언어 모델)을 사용하여 어떤 작업을 수행하고 그 결과를 반환하는 도구입니다. 이 때, 도구는 구글 검색, 데이터베이스 조회, Python REPL, 다른 체인 등과 같이 특정한 작업을 수행하는 함수를 의미합니다. 

LLM은 에이전트의 핵심인 언어 모델을 의미합니다. 

에이전트는 지원하는 에이전트 클래스를 참조하는 문자열입니다. 이 노트북에서는 가장 간단하고 Highest Level API를 사용하여 기본적으로 사용하는 방법에 대해 설명합니다.

In [10]:
GoogleSearch.SERP_API_KEY = os.getenv("SERPAPI_API_KEY")

In [11]:
# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 51.98ºf @12:56 AM Low: 50ºf @4:45 AM Approx. Precipitation / Rain Total: 0.063 in. 1hr. Precip / Rain Total (in.)
Thought: I need to use the calculator to raise the temperature to the .023 power.
Action: Calculator
Action Input: 51.98^.023
Observation: Answer: 1.0951263641624882
Thought: I now know the final answer.
Final Answer: The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0951263641624882.

> Finished chain.


'The high temperature in SF yesterday in Fahrenheit raised to the .023 power is 1.0951263641624882.'

# Memory: Add State to Chains and Agents

이전의 chain과 agent들은 모두 stateless하지만, 종종 이전 interaction에서 정보를 기억할 수 있도록 "memory"가 필요함

"short-term memory"은 이전 메세지를 기억함으로 더 나은 대화를 할 수 있으며,
"long-term memory"과 같이 시간이 지나더라도, 중요한 정보를 기억할 수 있습니다.

LangChain은 이러한 목적으로 특별히 만들어진 여러 chain과을 제공합니다. 이 노트북에서는 위의 두가지 유형의 memory를 위해 ConversationChain을 사용하는 방법을 설명합니다.

ConversationChain은 모든 이전 입력/출력을 기억하고, 전달되는 컨텍스트에 추가하는 간단한 유형의 메모리를 가지고 있습니다. 우리는 prompt를 볼 수 있도록 `verbose=True`로 설정하여 이 체인을 사용하는 방법을 살펴보겠습니다

In [12]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [13]:
output = conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.
 That's great! It's always nice to have a conversation with someone new. What would you like to talk about?


# Building a Language Model Application: Chat Models

LLM 대신 Chat models을 사용할 수 있다는 것을 설명하고 있습니다. Chat models은 언어 모델의 변형입니다. Chat models은 언어 모델을 내부적으로 사용하지만, 제공하는 인터페이스는 "텍스트 입력, 텍스트 출력" API가 아니라 "채팅 메시지"가 입력과 출력으로 제공됩니다. Chat model API는 상대적으로 새로운 기술이기 때문에, 적절한 추상화 방법을 아직 찾고 있습니다. 

## Get Message Completions from a Chat Model

In [14]:
chat = ChatOpenAI(temperature=0)

In [15]:
chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'aime programmer.", additional_kwargs={})

In [16]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="Translate this sentence from English to French. I love programming."),
]
chat(messages)
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'aime programmer.", additional_kwargs={})

In [17]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="Translate this sentence from English to French. I love programming."),
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="Translate this sentence from English to French. I love artificial intelligence."),
    ],
]
result = chat.generate(batch_messages)
result
# -> LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text="J'aime l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'aime l'intelligence artificielle.", additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 71, 'completion_tokens': 18, 'total_tokens': 89}})

LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text="J'aime l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'aime l'intelligence artificielle.", additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 16, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo'})

In [18]:
result.llm_output["token_usage"]
# -> {'prompt_tokens': 71, 'completion_tokens': 18, 'total_tokens': 89}

{'prompt_tokens': 73, 'completion_tokens': 16, 'total_tokens': 89}

## Chat Prompt Templates

`ChatPromptTemplate`은 하나 이상의 `MessagePromptTemplate`으로부터 구성될 수 있으며, `format_prompt`를 사용하여 `PromptValue`를 반환할 수 있습니다. `PromptValue`는 llm 또는 chat 모델의 입력으로 사용할 수 있는 문자열 또는 Message 객체로 변환할 수 있습니다. 또한, 템플릿에서는 `from_template` 메서드를 사용하여 편리하게 `ChatPromptTemplate`을 만들 수 있습니다.

In [19]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English", output_language="French", text="I love programming."
    ).to_messages()
)
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'adore la programmation.", additional_kwargs={})

## Chains with Chat Models

In [20]:
chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="French", text="I love programming.")
# -> "J'aime programmer."

"J'adore la programmation."

## Agents with Chat Model

In [21]:
# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```

Observation: singer Harry Styles
Thought:Now I need to find Harry Styles' age and raise it to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Harry Styles age"
}
```


Observation: 29 years
Thought:Now I need to calculate 29 raised to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "29^0.23"
}
```


Observation: Answer: 2.169459462491557
Thought:I now know the final answer.
Final Answer: 2.169459462491557

> Finished chain.


'2.169459462491557'

## Memory: Add State to Chains and Agents

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")
# -> 'Hello! How can I assist you today?'


conversation.predict(input="I'm doing well! Just having a conversation with an AI.")
# -> "That sounds like fun! I'm happy to chat with you. Is there anything specific you'd like to talk about?"

conversation.predict(input="Tell me about yourself.")
# -> "Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations like this one. Is there anything else you'd like to know about me?"

"Sure! I am an AI language model designed to assist with various tasks such as answering questions, generating text, and providing recommendations. I was created using natural language processing techniques and machine learning algorithms, which allow me to understand and respond to human language. I am constantly learning and improving based on the interactions I have with users like you. Is there anything else you'd like to know?"